# Rede de citação
Dados do dataset 2020 que tenham relação com pelo menos uma das vacinas Coronavac,Astrazeneca,Moderna,Janssen e Pfizer.

In [2]:
# Instalar pycairo, igraph e depois importar bibliotecas
import igraph as ig
import numpy as np
import os
import json
import pandas as pd 
import csv
import math
from igraph import Graph, plot
import random

In [2]:
df = pd.read_csv("..\Jupyter Notebook\COVID19Dataset\metadata.csv", low_memory=False)

In [3]:
# Cria uma cópia do DataFrame original e remove linhas onde 'abstract' é nulo
df_notna_aux1 = df.copy()
df_notna_aux1 = df_notna_aux1[df_notna_aux1['abstract'].notna()]

# Remove linhas onde 'authors' é nulo
df_notna_aux2 = df_notna_aux1.copy()
df_notna_aux2 = df_notna_aux2[df_notna_aux2['authors'].notna()]

# Remove linhas onde 'publish_time' é nulo
df_notna_aux3 = df_notna_aux2.copy()
df_notna_aux3 = df_notna_aux3[df_notna_aux3['publish_time'].notna()]

# Remove linhas onde 'pdf_json_files' é nulo
df_notna = df_notna_aux3.copy()
df_notna = df_notna[df_notna['pdf_json_files'].notna()]

# Converte o formato de 'publish_time' para datetime
df_notna['publish_time'] = pd.to_datetime(df_notna['publish_time'])

# Filtra o ano de 2020
df_ano = df_notna[(df_notna['publish_time'] >= '2020-01-01 00:00:00') & (df_notna['publish_time'] <= '2021-01-01 00:00:00')]

# Filtra por nomes de pelo menos uma das vacinas
df_vacina = df_ano[df_ano['abstract'].str.contains("Coronavac|Astrazeneca|Moderna|Janssen|Pfizer", case=False)]

# Filtra os artigos onde o campo 'title' não é nulo
df_finalizado = df_vacina[df_vacina['title'].notna()]

print("Número de artigos DFFinalizado:", len(df_finalizado))
print("Número de artigos DFNotNA:", len(df_notna))

Número de artigos DFFinalizado: 66
Número de artigos DFNotNA: 325855


In [4]:
#Aqui é feita a união de caminho de diretório de forma portável, armazenando os caminhos completos em uma lista
datafiles = []
for filename in df_finalizado['pdf_json_files']:
    ifile = os.path.join('..\Jupyter Notebook\COVID19Dataset', filename)
    datafiles.append(ifile)
    
with open(datafiles[0],'r')as f:
    doc = json.load(f)

In [5]:
citbib = []

for file in datafiles:
    try:
        with open(file, 'r') as f:
            doc = json.load(f)
        
        id = doc['paper_id']
        title = doc['metadata']['title']
        
        authors_doc = []
        for author in doc['metadata']['authors']:
            first = author['first']
            middle = author.get('middle', None)
            last = author['last']
            suffix = author.get('suffix', None)
            
            affiliation = author.get('affiliation', {})
            laboratory = affiliation.get('laboratory', None)
            institution = affiliation.get('institution', None)
            
            location = affiliation.get('location', {})
            settlement = location.get('settlement', None)
            region = location.get('region', None)
            country = location.get('country', None)
            
            email = author.get('email', None)
            
            authors_doc.append({
                'first': first,
                'middle': middle,
                'last': last,
                'suffix': suffix,
                'affiliation': {
                    'laboratory': laboratory,
                    'institution': institution,
                    'location': {
                        'settlement': settlement,
                        'region': region,
                        'country': country
                    }
                },
                'email': email
            })
        
        bibEntries = []
        for key, value in doc['bib_entries'].items():
            refid = key
            bib_title = value['title']
            
            authors_bib = []
            for author in value['authors']:
                first = author['first']
                middle = author.get('middle', None)
                last = author['last']
                suffix = author.get('suffix', None)
                
                authors_bib.append({
                    'first': first,
                    'middle': middle,
                    'last': last,
                    'suffix': suffix
                })
            
            year = value['year']
            
            try:
                DOI = value['other_ids']['DOI'][0]
            except KeyError:
                DOI = np.nan
            
            bibEntries.append({
                'refid_bib': refid,
                'title': bib_title,
                'authors_bib': authors_bib,
                'year': year,
                'DOI': DOI
            })
        
        citbib.append({
            'id': id,
            'title': title,
            'authors_doc': authors_doc,
            'bib': bibEntries
        })
    except:
        pass

In [6]:
#salvando em um csv
df = pd.DataFrame(citbib)
df.to_csv('citbib.csv', index=False)

In [7]:
#Função para remover caracteres desconhecidos
def remove_unidentified_chars(text):
    try:
        text.encode('utf-8').decode('utf-8')
        return text
    except UnicodeDecodeError:
        return ''

In [8]:
#Mapeamento de cada titulo para um ID único, já que os ids das citações se repetem para cada artigo, além da remoção de caracteres desconhecidos.
title_mapping = {}
current_id = 0

for entry in citbib:
    #remover caracteres não identificados
    article_title = remove_unidentified_chars(entry['title']).strip()
    if article_title: #verificação se não ficou vazio
        if article_title not in title_mapping:
            title_mapping[article_title] = current_id
            current_id += 1

    for bib_entry in entry['bib']:
        bib_title = remove_unidentified_chars(bib_entry['title']).strip()
        if bib_title:
            if bib_title not in title_mapping:
                title_mapping[bib_title] = current_id
                current_id += 1

#Salvando em um csv com "id" e "title" para futuras consultas
with open('mapeamento.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['id', 'title']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    
    for title, unique_id in title_mapping.items():
        writer.writerow({'id': unique_id, 'title': title})

In [9]:
#Percorrendo citbib e olhando os IDs de "mapeamento", relaciona o id do artigo com o idcitado.
citations = {}

for entry in citbib:
    article_title = remove_unidentified_chars(entry['title']).strip()
    if article_title:
        title_id = title_mapping.get(article_title)
        if title_id is not None:
            if title_id not in citations:
                citations[title_id] = set()  # set para não duplicar 

            for bib_entry in entry['bib']:
                bib_title = remove_unidentified_chars(bib_entry['title']).strip()
                if bib_title:
                    # Obter o ID da entrada de bibliografia a partir do mapeamento
                    bib_id = title_mapping.get(bib_title)
                    if bib_id is not None:
                        # Adicionar o ID da entrada de bibliografia ao conjunto de citações
                        citations[title_id].add(bib_id)  

# Salvar as citações em um arquivo CSV
with open('citations.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['idTitle', 'idCit']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    
    for title_id, bib_ids in citations.items():
        for bib_id in bib_ids:
            writer.writerow({'idTitle': title_id, 'idCit': bib_id})

In [3]:
title_mapping = {}
with open('mapeamento.csv', 'r', encoding='utf-8') as csvfile:  
    reader = csv.DictReader(csvfile)
    for row in reader:
        title_mapping[int(row['id'])] = row['title']

citations = []
with open('citations.csv', 'r', encoding='utf-8') as csvfile:  
    reader = csv.DictReader(csvfile)
    for row in reader:
        citations.append((int(row['idTitle']), int(row['idCit'])))

#Criando grafo direcionado, onde os nós são cada id em mapeamento e as ligações são as citações.
grafo = ig.Graph(directed=True)

grafo.add_vertices(len(title_mapping))

grafo.add_edges(citations)

#Atribuir label (imagem gerada é bem pesada)
grafo.vs["label"] = [title_mapping.get(node.index, "") for node in grafo.vs]

layout = grafo.layout_fruchterman_reingold()
#ig.plot(grafo, layout=layout)

In [4]:
#No grafo acima, existem muitos nós não citados, então é criado um grafo com a exclusão dos nós sem ligações
gCitacao = grafo.copy()

#Verifica nós sem conexões (citações) e remove
isolated_nodes = [v.index for v in gCitacao.vs if v.indegree() == 0 and v.outdegree() == 0]
gCitacao.delete_vertices(isolated_nodes)

layout = gCitacao.layout_fruchterman_reingold()
#ig.plot(gCitacao, layout=layout)

## Clusterização para atribuição de cores

In [25]:
color_palette = []

# Função para gerar uma cor aleatória em formato hexadecimal
def cor_aleatoria():
    r = random.randint(0, 255)
    g = random.randint(0, 255)
    b = random.randint(0, 255)
    return "#{:02X}{:02X}{:02X}".format(r, g, b)

# Gerar 185 cores aleatórias e adicioná-las à paleta (existem 183 nós no grafo de citações)
for _ in range(185):
    color_palette.append(cor_aleatoria())

In [26]:
#Algoritmo Walktrap para identificar os clusters no grafo direcionado
dendrogram = gCitacaoDegree.community_walktrap()

#Converter o dendrograma em um objeto de clustering
clusters = dendrogram.as_clustering()

#Obter a atribuição de cluster para cada nó
membership = clusters.membership

available_colors = color_palette.copy() 
cluster_to_color = {}

#Atribuir cada cor para um cluster diferente, somente repetindo as cores se todas acabarem
for cluster_num in set(membership):
    if len(available_colors) > 0:
        color = random.choice(available_colors)
        available_colors.remove(color)
    if len(available_colors) == 0:
        available_colors = color_palette
    cluster_to_color[cluster_num] = color

# Definir as cores dos nós com base nos clusters
gCitacaoDegree.vs["color"] = [cluster_to_color[membership[node.index]] for node in gCitacaoDegree.vs]

#Resultado
layout = gCitacaoDegree.layout_fruchterman_reingold()
#ig.plot(gCitacao, layout=layout)

## Betweenness

In [14]:
gCitacaoBetweenness = gCitacao.copy()
# Calcule o betweenness
betweenness = gCitacaoBetweenness.betweenness(vertices=None, directed=True, cutoff=None, weights=None)

# Calcule o máximo e o mínimo do betweenness
max_betweenness = max(betweenness)
min_betweenness = min(betweenness)
print("Betweenness Mínimo:", min_betweenness)
print("Betweenness Máximo:", max_betweenness)

# Aplique a escala de 0 a 1 ao betweenness
scaled_betweenness = [(b - min_betweenness) / (max_betweenness - min_betweenness) for b in betweenness]

Betweenness Mínimo: 0.0
Betweenness Máximo: 148.0


In [15]:
# Define o tamanho dos nós com base no betweenness escalado e calcula os tamanhos dos nós com escala logarítmica
gCitacaoBetweenness.vs["size"] = [2.5 + 10 * b for b in scaled_betweenness]

layout = gCitacaoBetweenness.layout_fruchterman_reingold()
#plot(gCitacaoBetweenness, layout=layout)

## Degree

In [22]:
gCitacaoDegree = gCitacao.copy()
in_degrees = gCitacaoDegree.indegree()

In [23]:
in_degrees = gCitacaoDegree.indegree()

# Defina o tamanho dos vértices com base nos graus de entrada usando log
gCitacaoDegree.vs["size"] = [math.log(in_degree) * 3 if in_degree > 0 else 0 for in_degree in in_degrees]

layout = gCitacaoDegree.layout_fruchterman_reingold()
#plot(gCitacao, layout=layout)